<a href="https://colab.research.google.com/github/sivatanujgunta/AItriesTrading/blob/main/AItriesTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Thanks to ChatGPT and Gemini AI for making vibe coding easy

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [3]:
# Feature calculation parameters
short_term_ema_period = 10  # Short-term Exponential Moving Average period
long_term_ema_period = 50  # Long-term Exponential Moving Average period
atr_period = 14  # Average True Range period for stop loss calculation
bollinger_band_sma_period = 20  # Bollinger Band Simple Moving Average period
bollinger_band_std_multiplier = 2  # Bollinger Band standard deviation multiplier

In [4]:
# Fetch historical data for a stock
symbol = 'AAPL'  # You can replace this with any stock ticker
data = yf.download(symbol, start='2020-01-01', end='2025-01-01')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
# Calculate the short-term and long-term Exponential Moving Averages (EMAs)
data['EMA10'] = data['Close'].ewm(span=short_term_ema_period, adjust=False).mean()
data['EMA50'] = data['Close'].ewm(span=long_term_ema_period, adjust=False).mean()

In [6]:
# Calculate the Relative Strength Index (RSI)
delta = data['Close'].diff(1)
gain = (delta.where(delta > 0, 0)).rolling(window=atr_period).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=atr_period).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))

In [7]:
# Calculate the Average True Range (ATR) for setting the stop loss
high_low = data['High'] - data['Low']
high_close = np.abs(data['High'] - data['Close'].shift(1))
low_close = np.abs(data['Low'] - data['Close'].shift(1))
tr = pd.concat([high_low, high_close, low_close], axis=1, ignore_index=True)
data['ATR'] = tr.max(axis=1).rolling(window=atr_period).mean()

In [8]:
# Calculate Bollinger Bands for Close prices
data['SMA20'] = data['Close'].rolling(window=bollinger_band_sma_period).mean()  # Simple Moving Average
data['STD20'] = data['Close'].rolling(window=bollinger_band_sma_period).std()   # Standard Deviation
data['Upper_Band'] = data['SMA20'] + (bollinger_band_std_multiplier * data['STD20'])  # Upper Bollinger Band
data['Lower_Band'] = pd.DataFrame(data['SMA20'] - (bollinger_band_std_multiplier * data['STD20']))  # Lower Bollinger Band
data['Lower_Band']

,Lower_Band
Date,
2020-01-02,NaN
2020-01-03,NaN
2020-01-06,NaN
2020-01-07,NaN
2020-01-08,NaN
...,...
2024-12-24,232.940509
2024-12-26,234.080573
2024-12-27,236.053844


In [9]:
# Generate Buy and Sell signals

# Strategy 1 - Short term and Long term EMA crossover
data['Buy_Signal'] = np.where(data['EMA10'] > data['EMA50'], 1, 0)  # Buy signal when EMA10 > EMA50
data['Sell_Signal'] = np.where(data['EMA10'] < data['EMA50'], -1, 0)  # Sell signal when EMA10 < EMA50

# Strategy 2 - Short term and Long term EMA crossover with RSI filter
rsi_oversold_threshold = 30
rsi_overbought_threshold = 70
data['Buy_Signal'] = np.where((data['EMA10'] > data['EMA50']) & (data['RSI'] > rsi_oversold_threshold), 1, 0)
data['Sell_Signal'] = np.where((data['EMA10'] < data['EMA50']) | (data['RSI'] < rsi_overbought_threshold), -1, 0)

# Strategy 3 - Bollinger bands with RSI filter
print(type(data['Close']))
print(type(data['SMA20']))
print(type(data['Lower_Band']))
data['Close'], data['Lower_Band'] = data['Close'].align(data['Lower_Band'], axis=1, copy=False)
data['Buy_Signal'] = np.where((data['Close'] <= data['Lower_Band']) & (data['RSI'] > rsi_oversold_threshold), 1, 0)
data['Sell_Signal'] = np.where((data['Close'] >= data['Upper_Band']) & (data['RSI'] < rsi_overbought_threshold), -1, 0)

data['Signal'] = data['Buy_Signal'] + data['Sell_Signal']

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


ValueError: Columns must be same length as key

In [ ]:
# Trading parameters
initial_balance = 10000  # Starting balance
risk_per_trade = 0.02  # Risk 2% of the balance per trade
risk_to_reward = 2  # Risk to reward ratio of 1:2 for stop loss : take profit
stop_loss_factor = 1.5  # Stop loss factor (1.5x ATR distance)
take_profit_factor = risk_to_reward * stop_loss_factor  # Take profit distance

In [ ]:
# Simulate trading
balance = initial_balance
positions = 0
buy_price = 0
stop_loss_price = 0
take_profit_price = 0
trade_log = []

for i in range(atr_period, len(data)):
    close = data['Close'].iloc[i].item()
    # Check if a buy signal is generated
    if (data['Signal'].iloc[i] == 1) and positions == 0:
        # Calculate position size based on risk per trade
        atr = data['ATR'].iloc[i]
        position_size = (balance * risk_per_trade / (atr * stop_loss_factor))

        # Open a position
        positions = position_size
        trade_log.append(f"Buy {position_size} shares at {buy_price} on {data.index[i]}")

        # Set buy price and stop loss
        buy_price = close
        stop_loss_price = buy_price - (atr * stop_loss_factor)
        take_profit_price = buy_price + (atr * take_profit_factor)

    # Check if a sell signal is generated and we have an open position
    elif data['Signal'].iloc[i] == -1 and positions > 0:
        # Close position
        sell_price = close
        profit = (sell_price - buy_price) * positions
        balance += profit
        positions = 0  # Close position
        trade_log.append(f"Sell {position_size} shares at {sell_price} on {data.index[i]} (Profit: {profit})")

    # Check if stop loss is hit
    elif positions > 0 and close <= stop_loss_price:
        # Close position at stop loss
        sell_price = close
        loss = (sell_price - buy_price) * positions
        balance += loss
        positions = 0  # Close position
        trade_log.append(f"Stop Loss hit for {position_size} shares at {sell_price} on {data.index[i]} (Loss: {loss})")

    # Check if take profit is hit
    elif positions > 0 and close >= take_profit_price:
        # Close position at take profit
        sell_price = close
        profit = (sell_price - buy_price) * positions
        balance += profit
        positions = 0  # Close position
        trade_log.append(f"Take Profit hit for {position_size} shares at {sell_price} on {data.index[i]} (Profit: {profit})")


In [ ]:
# Output results
final_balance = balance
print(f"Final Balance: {final_balance}")
print(f"Total profit/loss: {final_balance - initial_balance}")

In [ ]:
# Print trade log
for trade in trade_log:
    print(trade)

In [ ]:
# Plot the price with buy and sell signals
plt.figure(figsize=(12, 6))
plt.plot(data['Close'], label='Price')
plt.plot(data['EMA10'], label='EMA 10', linestyle='--')
plt.plot(data['EMA50'], label='EMA 50', linestyle='--')
plt.scatter(data.index[data['Signal'] == 1], data['Close'][data['Signal'] == 1], marker='^', color='g', label='Buy Signal')
plt.scatter(data.index[data['Signal'] == -1], data['Close'][data['Signal'] == -1], marker='v', color='r', label='Sell Signal')
plt.title(f'{symbol} Moving Average Crossover Strategy')
plt.legend()
plt.show()

In [ ]:
# prompt: tell some parameters to measure the strategy performance

# Calculate additional performance metrics
total_trades = len([trade for trade in trade_log if "Profit" in trade or "Loss" in trade])
winning_trades = len([trade for trade in trade_log if "Profit" in trade])
losing_trades = len([trade for trade in trade_log if "Loss" in trade])

if total_trades > 0:
    win_rate = winning_trades / total_trades
    max_drawdown = 0  # Placeholder for max drawdown calculation
    # ... (Code to calculate max drawdown - see explanation below)
    average_win = np.mean([float(trade.split("(")[1].split(":")[1].split(")")[0]) for trade in trade_log if "Profit" in trade]) if winning_trades else 0
    average_loss = np.mean([abs(float(trade.split("(")[1].split(":")[1].split(")")[0])) for trade in trade_log if "Loss" in trade]) if losing_trades else 0
    profit_factor = (sum(float(trade.split("(")[1].split(":")[1].split(")")[0]) for trade in trade_log if "Profit" in trade) /
                     abs(sum(float(trade.split("(")[1].split(":")[1].split(")")[0]) for trade in trade_log if "Loss" in trade))) if losing_trades else 0

    print(f"Total Trades: {total_trades}")
    print(f"Win Rate: {win_rate:.2f}")
    print(f"Max Drawdown: {max_drawdown:.2f}")  # Placeholder, needs calculation
    print(f"Average Win: {average_win:.2f}")
    print(f"Average Loss: {average_loss:.2f}")
    print(f"Profit Factor: {profit_factor:.2f}")
else:
    print("No trades executed.")
